In [ ]:
using Pkg,Statistics,Random,LinearAlgebra,Printf,GZip,Knet,Images,Distributions,Plots #dy: do you need all this?

In [ ]:
L=31; # number of training data points

In [ ]:
Range=2.0; # range of the x values

In [ ]:
NoiseRatio=0.3; # max noise/value

In [ ]:
function gen_noisy_gaussian(len;range=1.0,noiseratio=0.1)
    x = sort(randn(len)*range) #dy:randn does the same thing. # sort(rand(Normal(0,1),len)*range)
    y = exp.(-x.^2).*(-noiseratio*2*(rand(len).-0.5).+1) # fractional noise
    return (x,y)
end

In [ ]:
(x_train,y_train) = gen_noisy_gaussian(L,range=Range,noiseratio=NoiseRatio);

In [ ]:
plot(x_train,[y_train,exp.(-x_train.^2)])

In [ ]:
Layersize = 3; # number of neurons in a layer

In [ ]:
w = [0.1*rand(1,Layersize),0.1*rand(1,Layersize),0.1*rand(1,Layersize),0.1*rand(1,Layersize)]

In [ ]:
#dy: did you really mean to use .* here?  neural net layers use matmul. also biases are lower dim.
function loss(w,x,y)
    guesses = sum(tanh.(w[1].*x.+w[2]).*w[3] .+ w[4],dims=2) # w[1]=w, w[2]=w0, w[3]=w', w[4]=w0'
    return mean(abs2,y-guesses)
end

In [ ]:
x,y = x_train,y_train
sum(tanh.(w[1].*x.+w[2]).*w[3] .+ w[4],dims=2)
typeof(x_train)

In [ ]:
lossgradient = grad(loss)

In [ ]:
dw = lossgradient(w,x_train,y_train)

In [ ]:
@show w[1][1]
@show dw[1][1]
w[1][1] -= 0.1*dw[1][1]
@show w[1][1]

In [ ]:
function mytrain!(w;lr=0.1)
    dw = lossgradient(w,x_train,y_train);
    for i = 1:length(w)
        for j = 1:length(w[i])
            w[i][j] -= lr*dw[i][j];
        end
    end
    return w
end

In [ ]:
weights = [ deepcopy(mytrain!(w,lr=0.1)) for epoch=1:10 ]  # copy only copies the top layer, does not descend.